In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [20]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.int)

print(bits_info)

tensor([[1, 0, 0, 1],
        [1, 0, 0, 1],
        [1, 0, 1, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 0, 1, 1],
        [1, 1, 1, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 0],
        [1, 0, 1, 1],
        [1, 0, 1, 0],
        [1, 0, 1, 0],
        [1, 1, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 1],
        [1, 0, 0, 1],
        [0, 0, 0, 1],
        [1, 0, 1, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 0],
        [0, 0, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 0, 1],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [1, 1, 0, 0],
        [1, 1, 0, 1],
        [0, 1, 1, 1],
        [0, 1, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 1, 1],
        [0, 0, 1, 1],
        [0, 1, 1, 0],
        [1, 0, 1, 1],
        [1, 1, 1, 0],
        [1, 0, 1, 1],
        [0, 0, 0, 0],
        [0, 1, 0, 1],
        [1, 1, 0, 1],
        [0, 0, 0, 0],
        [1, 1, 0, 1],
        [1

Hamming(7,4) Encoder

In [21]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [54]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [1, 0, 0, 0, 1, 1, 0],
        [1, 0, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1],
        [0, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 1],
        [0, 1, 0, 0, 1, 0, 1],
        

BPSK Modulator + Noise

In [55]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        data: data received from the Hamming(7,4) encoder(Tensor)
        symbol_rate: Symbol rate in Hz
        carrier_freq: Carrier frequency in Hz
        snr_dB: Signal-to-noise ratio in dB

    Returns:
        time:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, snr_dB):
        
        data = torch.tensor(data, dtype=float)
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # # Time vector
            # time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
            # 
            # # Generate carrier signal
            # carrier = torch.cos(2 * torch.pi * carrier_freq * time)
            # 
            # # Modulate the signal
            # modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10),dtype=float)
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal
    
       
        return noise, data

In [56]:
data = encoded_codeword  # Binary data
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
noise, modulated_noise_signal = modulator(data, snr_dB)
print(modulated_noise_signal)
# print(noise)

tensor([[ 1.1356, -1.0319, -1.2670,  0.8640, -0.9235, -0.9398,  1.0116],
        [ 1.0726, -1.2415, -1.1227,  0.9281, -1.3855, -0.8948,  1.0067],
        [ 0.9800, -1.1354,  0.9820, -1.0719,  1.0190, -0.9455,  1.0727],
        [-0.8565, -0.8988, -1.2232, -1.1923, -0.8988, -0.6559, -0.9790],
        [ 1.1416,  1.0133, -1.0527, -0.9339, -0.9282,  1.1630,  1.2177],
        [-1.0642,  0.9368,  1.0492, -1.0012,  0.9867,  1.0068, -1.0920],
        [-0.9671, -1.1181,  0.9164,  0.7962,  0.8515, -1.2860, -0.9307],
        [ 0.7492,  1.0158,  0.9771,  1.0294,  1.1220,  0.9667,  1.0257],
        [-0.7444,  1.0905, -1.2242, -1.0384,  0.9787, -0.9226,  1.0915],
        [ 1.2335, -0.9258, -1.1032, -1.0344,  1.0428,  1.0016, -1.0926],
        [ 0.9146, -1.0622,  1.0636,  0.9424, -1.0770,  0.9649, -0.9931],
        [ 0.9237, -0.6969,  0.7901, -1.1115,  1.0164, -1.0504,  0.9822],
        [ 0.6927, -1.0953,  0.9389, -0.9557,  1.0981, -1.0574,  0.8030],
        [ 0.8477,  0.9518, -1.0035,  1.0569,  0.896

/var/folders/71/hry96qw555j_rlvg3tfgrh180000gn/T/ipykernel_36982/3378208061.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(data, dtype=float)


LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [57]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal.
        noise_std (float): Standard deviation of the noise.

    Returns:
        torch.Tensor: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))
    # likelihood_0 = 2 * (signal - 1) / noise_std**2
    # likelihood_1 = 2 * (signal + 1) / noise_std**2

    # Calculate the LLR
    # llr_values = likelihood_0 / likelihood_1
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [58]:
# received_signal = torch.tensor([2.6811e-01, -7.7690e-01, -3.2614e-01,  1.6319e+00, -1.5100e-01, -3.3976e-01,  8.3065e-01])
# LLR values: tensor([0.0170, -0.0491, -0.0206,  0.1032, -0.0096, -0.0215,  0.0525])

snr_dB = 15

llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[ 12.7725, -11.6055, -14.2499,   9.7174, -10.3866, -10.5699,  11.3777],
        [ 12.0634, -13.9633, -12.6273,  10.4386, -15.5828, -10.0640,  11.3219],
        [ 11.0217, -12.7697,  11.0442, -12.0559,  11.4604, -10.6340,  12.0642],
        [ -9.6327, -10.1089, -13.7571, -13.4097, -10.1083,  -7.3771, -11.0101],
        [ 12.8398,  11.3962, -11.8394, -10.5037, -10.4389,  13.0800,  13.6949],
        [-11.9686,  10.5355,  11.8002, -11.2607,  11.0974,  11.3236, -12.2820],
        [-10.8770, -12.5749,  10.3061,   8.9551,   9.5768, -14.4636, -10.4674],
        [  8.4262,  11.4250,  10.9895,  11.5777,  12.6187,  10.8725,  11.5361],
        [ -8.3725,  12.2649, -13.7688, -11.6785,  11.0069, -10.3761,  12.2758],
        [ 13.8726, -10.4121, -12.4074, -11.6336,  11.7286,  11.2649, -12.2883],
        [ 10.2866, -11.9462,  11.9616,  10.5993, -12.1125,  10.8521, -11.1693],
        [ 10.3882,  -7.8379,   8.8864, -12.5006,  11.4310, -11.8141,  11.0461],
        [  7.7912, -12.3181,

LDPC Decoder

In [146]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H, max_iter=50):
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.max_iter = max_iter
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr):
        for iteration in range(self.max_iter):
            # Variable node to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product0 = 0.5 * self.messages_v_to_c[connected_checks, j]
                    product = torch.prod(torch.tanh(product0))
                    # product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, i]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check node to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msg0 = self.messages_c_to_v[connected_vars, i]
                    sum_msgs = torch.sum(sum_msg0) - self.messages_v_to_c[j, i]
                    # sum_msgs = torch.sum(self.messages_v_to_c[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(0), torch.tensor(1))

        return estimated_bits
    
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])

llr_demodulator_output = llr_output
ldpc_bp = LDPCBeliefPropagation(H)
estimated_bits = ldpc_bp(llr_demodulator_output)

print("LLR Demodulator Output:", llr_demodulator_output)
print("Estimated Bits:", estimated_bits)

RuntimeError: expand(torch.FloatTensor{[7]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)

In [130]:
# class LDPCDecoder(nn.Module):
#     def __init__(self, H, num_iterations=10):
#         super(LDPCDecoder, self).__init__()
#         self.H = nn.Parameter(H, requires_grad=False)  # Parity-check matrix
#         self.num_iterations = num_iterations
#         
#     def ldpc_decode(self, llr_input):
#         # Your LDPC decoding algorithm (e.g., SPA or BP)
#         # ...
# 
#         return
# 
#     def forward(self, llr_input):
#         # Implement LDPC decoding algorithm
#         decoded = self.ldpc_decode(llr_input)
#         return decoded
# 
# 
# 
# def ldpc_loss(llr_hd, true):
#     
#     return 
#     
# 
# num_epochs = 20
# 
# # Set up your dataset (LLR values and corresponding codewords)
# # Initialize the LDPCDecoder model
# ldpc_decoder = LDPCDecoder(H)
# 
# # Define optimizer and learning rate
# optimizer = optim.Adam(ldpc_decoder.parameters(), lr=0.001)
# 
# # Training loop
# for epoch in range(num_epochs):
#     for llr_input, true_codeword in dataloader:
#         optimizer.zero_grad()
#         decoded = ldpc_decoder(llr_input)
#         loss = ldpc_loss(decoded, true_codeword) #cross entropy
#         loss.backward()
#         optimizer.step()
# 
#     # Print or log the loss for monitoring training progress
#     print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')


IndentationError: expected an indented block (2525148438.py, line 18)